In [ ]:
# https://github.com/DLR-RM/stable-baselines3/issues/383 (not so useful)
# https://github.com/DLR-RM/stable-baselines3/issues/1349

import sys
sys.path.append("/usr/src/gym4real")

In [ ]:
import torch as th
from stable_baselines3 import PPO
import onnx
import onnxruntime as rt
import numpy as np
import gymnasium as gym

### Convert Stable Baseline Model to ONNX

In [9]:
class OnnxablePolicyPyTorch2(th.nn.Module):
    def __init__(self, policy):
        super().__init__()
        self.policy = policy

    def forward(self, observation):
        # NOTE: Preprocessing is included, the only thing you need to do
        # is transpose the images if needed so that they are channel first
        # use deterministic=False if you want to export the stochastic policy
        return self.policy(observation, deterministic=False)

In [10]:
model_name = "Pretrained/ts1000000.zip"
model = PPO.load(model_name, device="cpu")

ModuleNotFoundError: No module named 'gym'

In [5]:
onnx_pytorch2 = OnnxablePolicyPyTorch2(model.policy)
observation_size = model.observation_space.shape
dummy_input = th.randn(1, *observation_size)

In [6]:
th.onnx.export(
    onnx_pytorch2,
    dummy_input,
    "model.onnx",
    opset_version=17,  # neeed a "updated" version of onnx
    input_names=["input"],
    verbose=0
)

============= Diagnostic Run torch.onnx.export version 2.0.1+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



### Load onxx Model

In [ ]:
providers = ['CUDAExecutionProvider', 'CPUExecutionProvider']
onnx_model = onnx.load(model_name)
onnx.checker.check_model(onnx_model)

ort_sess = rt.InferenceSession(model_name,providers=providers)